### Data Transformation

In [ ]:
from pathlib import Path
import os

# Fix working directory to project root regardless of where the notebook kernel started
project_root = Path.cwd()
if not (project_root / 'config' / 'config.yaml').exists():
    candidate = project_root.parent
    if (candidate / 'config' / 'config.yaml').exists():
        project_root = candidate

os.chdir(project_root)
print('cwd:', Path.cwd())

assert (Path.cwd() / 'config' / 'config.yaml').exists(), 'config/config.yaml not found in project root'


In [4]:
%pwd

'e:\\UDemy Final\\MLOPS\\TextSummarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path


In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        project_root = Path.cwd()
        if not (project_root / 'config' / 'config.yaml').exists():
            project_root = project_root.parent

        config_path = Path(config_path)
        params_filepath = Path(params_filepath)
        if not config_path.is_absolute():
            config_path = project_root / config_path
        if not params_filepath.is_absolute():
            params_filepath = project_root / params_filepath

        self.config=read_yaml(config_path)
        self.paramss=read_yaml(params_filepath)

        artifacts_root = Path(self.config.artifacts_root)
        if not artifacts_root.is_absolute():
            artifacts_root = project_root / artifacts_root
        create_directories([artifacts_root])

    def get_data_transformation_config(self)-> DataTransformationConfig:
        config=self.config.data_transformation

        project_root = Path.cwd()
        if not (project_root / 'config' / 'config.yaml').exists():
            project_root = project_root.parent

        root_dir = Path(config.root_dir)
        data_path = Path(config.data_path)
        if not root_dir.is_absolute():
            root_dir = project_root / root_dir
        if not data_path.is_absolute():
            data_path = project_root / data_path

        create_directories([root_dir])

        data_transformation_config=DataTransformationConfig(
            root_dir=root_dir,
            data_path=data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config



In [8]:
import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

### Data Transformation Component


In [ ]:
class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config=config
        self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

        target_encodings = self.tokenizer(text_target=example_batch['summary'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))


    

In [ ]:
# Hotfix for newer Transformers versions (replaces deprecated as_target_tokenizer flow)
def _convert_examples_to_features(self, example_batch):
    input_encodings = self.tokenizer(
        example_batch['dialogue'],
        max_length=1024,
        truncation=True
    )

    target_encodings = self.tokenizer(
        text_target=example_batch['summary'],
        max_length=128,
        truncation=True
    )

    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

DataTransformation.convert_examples_to_features = _convert_examples_to_features
print('DataTransformation hotfix applied')


In [ ]:
config=ConfigurationManager()
data_transformation_config=config.get_data_transformation_config()

project_root = Path('/Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer')
data_transformation_config.data_path = project_root / 'artifacts' / 'data_ingestion' / 'samsum_dataset'
data_transformation_config.root_dir = project_root / 'artifacts' / 'data_transformation'

print(data_transformation_config.data_path)
print(data_transformation_config.data_path.exists())

data_transformation=DataTransformation(config=data_transformation_config)
data_transformation.convert()
